# Análise Comparativa de Métodos de Validação Cruzada no Desempenho do Classificador SVM para Predição de Doenças Cardíacas

Este notebook apresenta uma análise comparativa de diferentes métodos de validação cruzada aplicados a um classificador SVM (Support Vector Machine) para predição de doenças.

## Objetivos

1. Implementar e comparar diferentes métodos de validação cruzada:
   - K-Fold Cross-Validation
   - Stratified K-Fold Cross-Validation
   - Leave-One-Out Cross-Validation (opcional)

2. Avaliar o desempenho do classificador SVM usando múltiplas métricas:
   - Acurácia
   - Precisão
   - Recall
   - F1-Score

3. Visualizar e comparar os resultados obtidos

## 1. Importação de Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import (
    cross_val_score, 
    KFold, 
    StratifiedKFold, 
    LeaveOneOut,
    cross_validate
)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    make_scorer,
    confusion_matrix,
    classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Bibliotecas importadas com sucesso!")

## 2. Carregamento e Exploração dos Dados

In [ ]:
# Carrega o dataset
# Utilizamos o dataset de câncer de mama como proxy para análise de doenças
data = load_breast_cancer()
X = data.data
y = data.target

# Informações básicas
print(f"Dimensões do dataset: {X.shape}")
print(f"Número de amostras: {X.shape[0]}")
print(f"Número de características: {X.shape[1]}")
print(f"\nDistribuição de classes:")
unique, counts = np.unique(y, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Classe {label}: {count} amostras ({count/len(y)*100:.2f}%)")

# Normalização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nDados normalizados com sucesso!")

## 3. Implementação dos Métodos de Validação Cruzada

### 3.1 K-Fold Cross-Validation

In [ ]:
def evaluate_kfold(X, y, n_splits=10, random_state=42):
    """
    Realiza K-Fold Cross-Validation.
    """
    print(f"\n{'='*60}")
    print(f"K-Fold Cross-Validation (k={n_splits})")
    print(f"{'='*60}")
    
    clf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=random_state)
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    scoring = {
        'accuracy': 'accuracy',
        'precision': make_scorer(precision_score, zero_division=0),
        'recall': make_scorer(recall_score, zero_division=0),
        'f1': make_scorer(f1_score, zero_division=0)
    }
    
    scores = cross_validate(clf, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    
    results = {
        'method': f'K-Fold (k={n_splits})',
        'accuracy': scores['test_accuracy'],
        'precision': scores['test_precision'],
        'recall': scores['test_recall'],
        'f1': scores['test_f1']
    }
    
    # Imprime resultados
    for metric in ['accuracy', 'precision', 'recall', 'f1']:
        mean = results[metric].mean()
        std = results[metric].std()
        print(f"{metric.capitalize():12s}: {mean:.4f} (+/- {std:.4f})")
    
    return results

# Executa K-Fold com diferentes valores de k
kfold_5 = evaluate_kfold(X_scaled, y, n_splits=5)
kfold_10 = evaluate_kfold(X_scaled, y, n_splits=10)

### 3.2 Stratified K-Fold Cross-Validation

In [ ]:
def evaluate_stratified_kfold(X, y, n_splits=10, random_state=42):
    """
    Realiza Stratified K-Fold Cross-Validation.
    """
    print(f"\n{'='*60}")
    print(f"Stratified K-Fold Cross-Validation (k={n_splits})")
    print(f"{'='*60}")
    
    clf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=random_state)
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    scoring = {
        'accuracy': 'accuracy',
        'precision': make_scorer(precision_score, zero_division=0),
        'recall': make_scorer(recall_score, zero_division=0),
        'f1': make_scorer(f1_score, zero_division=0)
    }
    
    scores = cross_validate(clf, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    
    results = {
        'method': f'Stratified K-Fold (k={n_splits})',
        'accuracy': scores['test_accuracy'],
        'precision': scores['test_precision'],
        'recall': scores['test_recall'],
        'f1': scores['test_f1']
    }
    
    # Imprime resultados
    for metric in ['accuracy', 'precision', 'recall', 'f1']:
        mean = results[metric].mean()
        std = results[metric].std()
        print(f"{metric.capitalize():12s}: {mean:.4f} (+/- {std:.4f})")
    
    return results

# Executa Stratified K-Fold com diferentes valores de k
stratified_5 = evaluate_stratified_kfold(X_scaled, y, n_splits=5)
stratified_10 = evaluate_stratified_kfold(X_scaled, y, n_splits=10)

## 4. Comparação Visual dos Resultados

In [ ]:
# Prepara dados para visualização
all_results = {
    'K-Fold (k=5)': kfold_5,
    'K-Fold (k=10)': kfold_10,
    'Stratified K-Fold (k=5)': stratified_5,
    'Stratified K-Fold (k=10)': stratified_10
}

# Cria figura com múltiplos subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análise Comparativa de Métodos de Validação Cruzada\nClassificador SVM', 
             fontsize=16, fontweight='bold')

# 1. Comparação de Acurácia
ax1 = axes[0, 0]
methods = list(all_results.keys())
accuracies = [results['accuracy'].mean() for results in all_results.values()]
errors = [results['accuracy'].std() for results in all_results.values()]

bars = ax1.bar(range(len(methods)), accuracies, yerr=errors, 
               capsize=5, color='skyblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Método', fontweight='bold')
ax1.set_ylabel('Acurácia', fontweight='bold')
ax1.set_title('Comparação de Acurácia', fontweight='bold')
ax1.set_xticks(range(len(methods)))
ax1.set_xticklabels(methods, rotation=45, ha='right')
ax1.grid(axis='y', alpha=0.3)

for bar, val in zip(bars, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
            f'{val:.3f}', ha='center', va='bottom', fontsize=9)

# 2. Comparação de Múltiplas Métricas
ax2 = axes[0, 1]
x = np.arange(len(methods))
width = 0.2

metrics_data = {
    'Accuracy': [results['accuracy'].mean() for results in all_results.values()],
    'Precision': [results['precision'].mean() for results in all_results.values()],
    'Recall': [results['recall'].mean() for results in all_results.values()],
    'F1-Score': [results['f1'].mean() for results in all_results.values()]
}

colors = ['skyblue', 'lightgreen', 'lightcoral', 'plum']
for i, (metric, values) in enumerate(metrics_data.items()):
    offset = (i - 1.5) * width
    ax2.bar(x + offset, values, width, label=metric, 
           color=colors[i], edgecolor='black', alpha=0.7)

ax2.set_xlabel('Método', fontweight='bold')
ax2.set_ylabel('Score', fontweight='bold')
ax2.set_title('Comparação de Múltiplas Métricas', fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. Box Plot - Distribuição de Acurácia
ax3 = axes[1, 0]
accuracy_data = [results['accuracy'] for results in all_results.values()]
bp = ax3.boxplot(accuracy_data, labels=methods, patch_artist=True)

for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_alpha(0.7)

ax3.set_xlabel('Método', fontweight='bold')
ax3.set_ylabel('Acurácia', fontweight='bold')
ax3.set_title('Distribuição de Acurácia', fontweight='bold')
ax3.set_xticklabels(methods, rotation=45, ha='right')
ax3.grid(axis='y', alpha=0.3)

# 4. Heatmap - Comparação de Métricas
ax4 = axes[1, 1]
metrics_matrix = np.array([
    [results['accuracy'].mean(), results['precision'].mean(), 
     results['recall'].mean(), results['f1'].mean()]
    for results in all_results.values()
])

im = ax4.imshow(metrics_matrix.T, cmap='YlGnBu', aspect='auto')
ax4.set_xticks(range(len(methods)))
ax4.set_yticks(range(4))
ax4.set_xticklabels(methods, rotation=45, ha='right')
ax4.set_yticklabels(['Accuracy', 'Precision', 'Recall', 'F1-Score'])
ax4.set_title('Heatmap de Métricas', fontweight='bold')

# Adiciona valores no heatmap
for i in range(4):
    for j in range(len(methods)):
        text = ax4.text(j, i, f'{metrics_matrix[j, i]:.3f}',
                       ha="center", va="center", color="black", fontsize=9)

plt.colorbar(im, ax=ax4)

plt.tight_layout()
plt.show()

## 5. Análise Estatística Detalhada

In [ ]:
# Cria DataFrame com resultados
results_df = pd.DataFrame({
    'Método': methods,
    'Acurácia (média)': [results['accuracy'].mean() for results in all_results.values()],
    'Acurácia (std)': [results['accuracy'].std() for results in all_results.values()],
    'Precisão (média)': [results['precision'].mean() for results in all_results.values()],
    'Precisão (std)': [results['precision'].std() for results in all_results.values()],
    'Recall (média)': [results['recall'].mean() for results in all_results.values()],
    'Recall (std)': [results['recall'].std() for results in all_results.values()],
    'F1-Score (média)': [results['f1'].mean() for results in all_results.values()],
    'F1-Score (std)': [results['f1'].std() for results in all_results.values()]
})

print("\n" + "="*100)
print("RESUMO ESTATÍSTICO DOS RESULTADOS")
print("="*100)
print(results_df.to_string(index=False))

# Identifica o melhor método
best_idx = results_df['Acurácia (média)'].idxmax()
print(f"\n{'='*100}")
print(f"Melhor Método (Acurácia): {results_df.loc[best_idx, 'Método']}")
print(f"Acurácia: {results_df.loc[best_idx, 'Acurácia (média)']:.4f} (+/- {results_df.loc[best_idx, 'Acurácia (std)']:.4f})")
print(f"{'='*100}")

## 6. Conclusões

### Principais Observações:

1. **K-Fold vs Stratified K-Fold**: 
   - O Stratified K-Fold mantém a proporção de classes em cada fold
   - Especialmente importante para datasets desbalanceados
   - Geralmente oferece estimativas mais confiáveis

2. **Impacto do número de folds**:
   - Mais folds (k=10) geralmente resultam em estimativas mais estáveis
   - Menos folds (k=5) são computacionalmente mais eficientes
   - Trade-off entre viés e variância

3. **Variabilidade dos resultados**:
   - O desvio padrão indica a estabilidade do método
   - Métodos com menor desvio padrão são mais consistentes

4. **Escolha do método**:
   - Para datasets balanceados: K-Fold é suficiente
   - Para datasets desbalanceados: Prefira Stratified K-Fold
   - Para datasets pequenos: Considere Leave-One-Out (não implementado aqui por custo computacional)

## 7. Usando a Classe de Análise

Alternativamente, você pode usar a classe `SVMCrossValidationAnalysis` do módulo `svm_cv_analysis.py`:

In [ ]:
# from svm_cv_analysis import SVMCrossValidationAnalysis

# # Inicializa análise
# analysis = SVMCrossValidationAnalysis(random_state=42)

# # Carrega dados
# analysis.load_data()

# # Executa todos os métodos
# analysis.run_all_methods(k_values=[5, 10], include_loo=False)

# # Gera visualizações
# analysis.create_comparison_plots('cv_comparison.png')

# # Gera relatório
# analysis.generate_report()